In [ ]:
!pip install jpype1
!pip install konlpy
!pip install git+https://github.com/lovit/textrank.git

In [ ]:
!pip install transformers==2.2.0
!pip install spacy==2.0.12

In [ ]:
!pip install bert-extractive-summarizer==0.4.2 # in colab
# !pip install bert-extractive-summarizer

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon Dec 27 22:23:31 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 495.44       Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   44C    P0    29W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
from psutil import virtual_memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 27.3 gigabytes of available RAM

You are using a high-RAM runtime!


In [ ]:
import os
import json
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm

from konlpy.tag import Komoran
from textrank import KeysentenceSummarizer
from summarizer import Summarizer, TransformerSummarizer

In [ ]:
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

In [ ]:
BASE_DIR = "/content/drive/MyDrive/AI 부트캠프/Section Project/Codestates Project 1/data"
TRAIN_DIR = os.path.join(BASE_DIR, "train/")
VALID_DIR = os.path.join(BASE_DIR, "valid/")

In [ ]:
os.listdir(TRAIN_DIR), os.listdir(VALID_DIR)

(['newspaper_train.json', 'law_train.json', 'article_train.json'],
 ['newspaper_valid.json', 'article_valid.json', 'law_valid.json'])

In [ ]:
with open(TRAIN_DIR + "newspaper_train.json", "r", encoding="utf-8") as f:
    news_train = json.load(f)

for json_str in news_train:
    print(json_str)

name
delivery_date
documents


In [ ]:
with open(VALID_DIR + "newspaper_valid.json", "r", encoding="utf-8") as f:
    news_valid = json.load(f)

for json_str in news_valid:
    print(json_str)

name
delivery_date
documents


In [ ]:
def prepro_text(json_data, n):
    one = []
    item = json_data["documents"][n]

    one.append(item["category"])
    one.append(item["media_name"])
    one.append(item["media_type"])
    one.append(item["char_count"])

    full_temp = []
    for i in range(len(item["text"])):
        if item["text"][i]:
            for j in range(len(item["text"][i])):
                full_temp.append(item["text"][i][j]["sentence"])

    one.append(full_temp)
    one.append(" ".join([str(n) for n in item["extractive"]]))

    extract_temp = []
    for i in item["extractive"]:
        extract_temp.append(full_temp[i])
    
    one.append(" ".join(extract_temp))
    one.append(item["abstractive"][0])

    return one

In [ ]:
def json_to_df(json_data):
    df = pd.DataFrame(columns = ["category", "media_name", "media_type", "char_count", "full_text", "extractive_idx", "extractive", "abstractive"])

    #for i in tqdm(range(100)):
    for i in tqdm(range(len(json_data["documents"]))):
        try:
            df.loc[i] = prepro_text(json_data, i)
        except:
            print(i)
            pass

    return df

In [ ]:
train_df = json_to_df(news_train)

  0%|          | 0/243983 [00:00<?, ?it/s]

21810
50279
61894
67953
68553
184704


In [ ]:
train_df.head()

,category,media_name,media_type,char_count,full_text,extractive_idx,extractive,abstractive
0,종합,광양신문,online,927,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,2 3 10,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 벼를 심었던 논에 벼 대...
1,종합,광양신문,online,764,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",2 4 11,여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,여수시는 컬러빌리지 사업에 8억원을 투입하여 ‘색채와 빛’ 도시를 완성하여 고소천사...
2,스포츠,광양신문,online,1066,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,3 5 7,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남드래곤즈 임직원과 선수단이 4일 구봉산 정상에 올라 일출을 보며 2018년 구단...
3,정치,광양신문,online,746,"11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...",2 3 4,광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"광양시는 농업인들의 경쟁력을 높이고, 소득안정을 위해 매실·감·참다래 등 지역특화작..."
4,종합,광양신문,online,978,"홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",3 7 4,전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,올해 4월과 6월 두 차례에 걸쳐 타이완의 크루즈 관광객 4000여명이 여수에 입항...


In [ ]:
train_df.to_csv(BASE_DIR + "/prepro_train_all.tsv", sep="\t", index=False)

In [ ]:
valid_df = json_to_df(news_valid)

  0%|          | 0/30122 [00:00<?, ?it/s]

4318


In [ ]:
valid_df.head()

,category,media_name,media_type,char_count,full_text,extractive_idx,extractive,abstractive
0,정치,한국경제,online,1111,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",0 1 3,"[ 박재원 기자 ] '대한민국 5G 홍보대사'를 자처한 문재인 대통령은 ""넓고, 체...",8일 서울에서 열린 5G플러스 전략발표에 참석한 문재인 대통령은 5G는 대한민국 혁...
1,종합,한국경제,online,904,] 당 지도부 퇴진을 놓고 바른미래당 내홍이 격화되고 있다. 바른미래당이 8일 연 ...,2 1 6,바른정당계 하태경·이준석·권은희 최고위원은 지도부 총사퇴를 요구하며 회의를 보이콧했...,8일 바른미래당 최고의원 회의에 하태경 의원 등 5명의 최고의원이 지도부 퇴진을 요...
2,"IT,과학",한국경제,online,1104,[ 홍윤정 기자 ] 8일 서울 올림픽공원 K아트홀. 지난 3일 한국이 세계 최초로 ...,1 5 8,지난 3일 한국이 세계 최초로 5세대(5G) 이동통신 서비스를 상용화한 것을 기념하...,지난 3일 한국이 세계 첫 5세대 이동통신 서비스를 보편화한 것을 축하하는 '코리안...
3,사회,한국경제,online,740,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,0 1 2,] 박원순 서울시장(사진)이 8일 고층 재개발·재건축 관련 요구에 작심한 듯 쓴소리...,박원순 서울시장은 8일 서울시청에서 열린 '골목길 재생 시민 정책 대화'에 참석하여...
4,경제,한국경제,online,1389,"[ 임근호 기자 ] ""SK(주)와 미국 알파벳(구글 지주회사)의 간결한 지배구조를 ...",1 3 4,기업 지배구조 개선 등을 통해 높은 수익률을 올리는 것을 목표로 하는 공모펀드 'K...,주주가치 포커스를 운용하는 KB자산운용이 SK와 알파벳(구글 지주회사)의 모범적 ...


In [ ]:
valid_df.to_csv(BASE_DIR + "/prepro_valid_all.tsv", sep="\t", index=False)

## **test data**

In [ ]:
def temp_text(json_data, n):
    one = []
    item = json_data["documents"][n]

    full_temp = []
    for i in range(len(item["text"])):
        if item["text"][i]:
            for j in range(len(item["text"][i])):
                full_temp.append(item["text"][i][j]["sentence"])

    one.append(full_temp)
    one.append(" ".join(full_temp))

    extract_temp = []
    for i in item["extractive"]:
        extract_temp.append(full_temp[i])
    
    one.append(" ".join(extract_temp))

    return one

In [ ]:
def temp_to_df(json_data):
    df = pd.DataFrame(columns = ["textrank_text", "bert_text", "extractive"])

    for i in tqdm(range(30000)):
    #for i in tqdm(range(len(json_data["documents"]))):
        try:
            df.loc[i] = temp_text(json_data, i)
        except:
            print(i)
            pass

    return df

In [ ]:
temp_df = temp_to_df(news_train)

  0%|          | 0/30000 [00:00<?, ?it/s]

21810


In [ ]:
temp_df.reset_index(drop=True, inplace=True)
temp_df

,textrank_text,bert_text,extractive
0,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...
1,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...
2,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...
3,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...
4,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...","홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...
...,...,...,...
29994,"[오승현 기자, “K리그1 전 경기 고화질로 즐긴다”…SPOTV NOW, 6월부터 ...","오승현 기자 “K리그1 전 경기 고화질로 즐긴다”…SPOTV NOW, 6월부터 서비...",온라인 스포츠 플랫폼 스포티비 나우(SPOTV NOW)가 K리그1 전 경기 생중계에...
29995,"[오승현 기자, 한국전파진흥협회, ‘클라우드’&‘블록체인’ 취업연계 국비지원 교육과...","오승현 기자 한국전파진흥협회, ‘클라우드’&‘블록체인’ 취업연계 국비지원 교육과정 ...",특히 올해 2기를 맞이하는 클라우드&블록체인 교육과정은 4차 산업혁명 인프라의 근간...
29996,"[오승현 기자, 유통업계 새로운 바람, Z세대와 소통하라, 한국 코카-콜라사 제공,...","오승현 기자 유통업계 새로운 바람, Z세대와 소통하라 한국 코카-콜라사 제공 Z세대...","유통계는 이런 Z세대의 시선을 끌 수 있는 다양한 콘텐츠를 선보이는 한편, Z세대 ..."
29997,"[최연수, 여수시의회 웅천특위, 위원 구성 문제로 갈등 파행, 송하진 의원 “의장 ...","최연수 여수시의회 웅천특위, 위원 구성 문제로 갈등 파행 송하진 의원 “의장 독단으...",여수시의회 웅천택지개발사업 실태파악특별위원회(이하 웅천특위)가 위원 구성 문제로 갈...


## **TextRank**

In [ ]:
komoran = Komoran()

def komoran_tokenizer(sent):
    words = komoran.pos(sent, join=True)
    words = [w for w in words if ('/NN' in w or '/XR' in w or '/VA' in w or '/VV' in w)]
    return words

textrank_summarizer = KeysentenceSummarizer(
    tokenize = komoran_tokenizer,
    min_sim = 0.3,
    verbose = False
)

In [ ]:
textrank_sum = []
for i in tqdm(range(len(train_df))):
    text = train_df.loc[i, "full_text"]
    keysents = textrank_summarizer.summarize(text, topk=3)
    textrank_sum.append(keysents)

  0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
textrank_sum[0]

[(5,
  1.3926448869790695,
  '지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼 이외 다른 작물을 재배한 농업인이다.'),
 (6,
  1.2250787502366869,
  '지원 대상 작물은 1년생을 포함한 다년생의 모든 작물이 해당되나 재배 면적 확대 시 수급과잉이 우려되는 고추, 무, 배추, 인삼, 대파 등 수급 불안 품목은 제외된다.'),
 (3,
  1.1768735331083477,
  '쌀 생산조정제는 벼를 심었던 논에 벼 대신 사료작물이나 콩 등 다른 작물을 심으면 벼와의 일정 소득차를 보전해주는 제도다.')]

In [ ]:
textrank_res = []
for n in range(len(textrank_sum)):
    result = textrank_sum[n][0][2] + "\n" + textrank_sum[n][1][2] + "\n" + textrank_sum[n][2][2]
    textrank_res.append(result)

In [ ]:
'''
for i in range(185,29999):
    textrank_res.append(np.nan)
len(textrank_res)
'''

29999

In [ ]:
train_df["textrank_summary"] = textrank_res
train_df.head()

,textrank_text,bert_text,extractive,textrank_summary
0,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼...
1,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,시 관계자는“컬러빌리지 사업은 주민들의 적극적인 관심과 참여가 만들어낸 값진 성과물...
2,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남은 이어 측면 공격력 강화를 위해 포항에서 활약한 완델손을 1년 임대 영입했다....
3,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"교육은 시의 특화작목인 감, 참다래, 생강, 매실 등에 대한 저비용 생산과 고품질 ..."
4,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...","홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,"전남도 관계자는“중국 중심의 크루즈 유치에서 일본, 타이완 등 시장 다변화 노력의 ..."


## **bert extractive summarization**

In [ ]:
bert_model = Summarizer()
bert_temp = []
for i in tqdm(range(len(train_df))):
    try:
        bert_text = train_df.loc[i,"full_text"]
        bert_summary = ''.join(bert_model(" ".join(bert_text), min_length=60))
        bert_temp.append(bert_summary)
    except:
        bert_temp.append(np.nan)
        pass

  0%|          | 0/29999 [00:00<?, ?it/s]

In [ ]:
train_df["bert_summary"] = bert_temp
train_df

,textrank_text,bert_text,extractive,textrank_summary,bert_summary
0,"[ha당 조사료 400만원…작물별 차등 지원, 이성훈 sinawi@hanmail.n...",ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...,전라남도가 쌀 과잉문제를 근본적으로 해결하기 위해 올해부터 시행하는 쌀 생산조정제를...,지원사업 대상은 2017년산 쌀 변동직불금을 받은 농지에 10a(300평) 이상 벼...,ha당 조사료 400만원…작물별 차등 지원 이성훈 sinawi@hanmail.net...
1,"[8억 투입, 고소천사벽화·자산마을에 색채 입혀, 이성훈 sinawi@hanmail...","8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n...",여수시는 원도심 일대에서 추진된 컬러빌리지 사업을 지난해 말 마무리하며 색채와 빛의...,시 관계자는“컬러빌리지 사업은 주민들의 적극적인 관심과 참여가 만들어낸 값진 성과물...,"8억 투입, 고소천사벽화·자산마을에 색채 입혀 이성훈 sinawi@hanmail.n..."
2,"[전남드래곤즈 해맞이 다짐…선수 영입 활발, 이성훈 sinawi@hanmail.ne...",전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...,임직원과 선수단 모두는 이날 구봉산 정상에 올라 일출을 보며 2018년 구단 목표를...,전남은 이어 측면 공격력 강화를 위해 포항에서 활약한 완델손을 1년 임대 영입했다....,전남드래곤즈 해맞이 다짐…선수 영입 활발 이성훈 sinawi@hanmail.net ...
3,"[11~24일, 매실·감·참다래 등 지역특화작목, 이성훈 sinawi@hanmail...","11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n...",광양시는 오는 11일부터 24일까지 농업인교육관과 읍면동 회의실에서 농업인 1050...,"교육은 시의 특화작목인 감, 참다래, 생강, 매실 등에 대한 저비용 생산과 고품질 ...","11~24일, 매실·감·참다래 등 지역특화작목 이성훈 sinawi@hanmail.n..."
4,"[홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항, 이성훈 sinawi@han...","홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma...",전라남도는 올해 4월과 6월 홍콩 크루즈선사 스타크루즈의 5만톤급‘아쿠아리우스’호가...,"전남도 관계자는“중국 중심의 크루즈 유치에서 일본, 타이완 등 시장 다변화 노력의 ...","홍콩 크루즈선사‘아쿠아리우스’ 4, 6월 여수항 입항 이성훈 sinawi@hanma..."
...,...,...,...,...,...
29994,"[오승현 기자, “K리그1 전 경기 고화질로 즐긴다”…SPOTV NOW, 6월부터 ...","오승현 기자 “K리그1 전 경기 고화질로 즐긴다”…SPOTV NOW, 6월부터 서비...",온라인 스포츠 플랫폼 스포티비 나우(SPOTV NOW)가 K리그1 전 경기 생중계에...,NaN,NaN
29995,"[오승현 기자, 한국전파진흥협회, ‘클라우드’&‘블록체인’ 취업연계 국비지원 교육과...","오승현 기자 한국전파진흥협회, ‘클라우드’&‘블록체인’ 취업연계 국비지원 교육과정 ...",특히 올해 2기를 맞이하는 클라우드&블록체인 교육과정은 4차 산업혁명 인프라의 근간...,NaN,NaN
29996,"[오승현 기자, 유통업계 새로운 바람, Z세대와 소통하라, 한국 코카-콜라사 제공,...","오승현 기자 유통업계 새로운 바람, Z세대와 소통하라 한국 코카-콜라사 제공 Z세대...","유통계는 이런 Z세대의 시선을 끌 수 있는 다양한 콘텐츠를 선보이는 한편, Z세대 ...",NaN,NaN
29997,"[최연수, 여수시의회 웅천특위, 위원 구성 문제로 갈등 파행, 송하진 의원 “의장 ...","최연수 여수시의회 웅천특위, 위원 구성 문제로 갈등 파행 송하진 의원 “의장 독단으...",여수시의회 웅천택지개발사업 실태파악특별위원회(이하 웅천특위)가 위원 구성 문제로 갈...,NaN,NaN


In [ ]:
train_df["bert_summary"].isnull().sum()

29925